Импорт библиотек

In [1]:
import pandas as pd
import requests
import time
import json
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime

Автоматизацию можно настроить через Apache Airflow или через Cron. Для простоты сделал локально через планировщик Windows.

Скорее всего в будущем переделаю схему: выгрузка с API и трансформация во временную таблицу, подключение к БД и insert в основную таблицу

#Выгрузка

In [33]:
# Путь к файлу
filepath  = Path('/content/drive/MyDrive/df.csv')

In [278]:
# Загрузка файла
df = pd.read_csv(filepath)

In [279]:
df.shape

(6, 14)

In [280]:
df.head(1)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,6928268805,5810066641,False,1671561300,773,6029.0,2.0,7,22,81,3,273,"59,57,15,3,77","38,135,47,119,89"


In [281]:
# API запрос с переводом json'а в датафрейм
j = requests.get(f'https://api.opendota.com/api/publicMatches')
publicMatches = j.json()
df_publicmatches = pd.DataFrame.from_dict(publicMatches)

In [282]:
# Фильтрация высокоранговых матчей. Проверка количетсва строк
df_publicmatches_hight_mmr = df_publicmatches.query('avg_rank_tier == 81')
df_publicmatches_hight_mmr.shape

(4, 14)

In [283]:
# Присоединения новых строк к датафрейму. Проверка количества строк
frames = [df, df_publicmatches_hight_mmr]
df = pd.concat(frames)
df.shape

(10, 14)

In [284]:
# Создание массива и датафрейма с уникальными match_id для более сложного API запроса с более полной статистикой по матчу. Требует доработки
# matches_array = df.match_id.unique()
# matches_array = pd.DataFrame.from_dict(matches_array)
# df_match_id = matches_array.rename(columns={ 0 : "match_id"})
# df_match_id.shape

In [285]:
# Так как API запрос парсит недавние 100 матчей, рандомно. То могут встречаться дубликаты. Избавимся от них
df = df.drop_duplicates(subset=['match_id'])

In [286]:
# Сохранение 
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath, index=False)

#Трансформация

In [287]:
# Перевод в формат времени
# df['start_time'] = pd.to_datetime(df['start_time'], unit='s')
# df['duration'] = pd.to_datetime(df['duration'], unit="s").dt.time

In [288]:
# Сплит колонок
# df[['dtp_1', 'dtp_2', 'dtp_3', 'dtp_4', 'dtp_5']] = df['dire_team']. str.split(',' , expand= True )
# df[['rtp_1', 'rtp_2', 'rtp_3', 'rtp_4', 'rtp_5']] = df['radiant_team']. str.split(',' , expand= True )

# Загрузка

In [289]:
# Подключение к БД
# from sqlalchemy import create_engine
# con = create_engine()

In [290]:
# Ускорение выгрузки
# import csv
# from io import StringIO

# def psql_insert_copy(table, conn, keys, data_iter):
#     # gets a DBAPI connection that can provide a cursor
#     dbapi_conn = conn.connection
#     with dbapi_conn.cursor() as cur:
#         s_buf = StringIO()
#         writer = csv.writer(s_buf)
#         writer.writerows(data_iter)
#         s_buf.seek(0)

#         columns = ', '.join('"{}"'.format(k) for k in keys)
#         if table.schema:
#             table_name = '{}.{}'.format(table.schema, table.name)
#         else:
#             table_name = table.name

#         sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
#             table_name, columns)
#         cur.copy_expert(sql=sql, file=s_buf)

In [291]:
# df.to_sql('matches_stat', con, index=False, if_exists='replace', method=psql_insert_copy)